In [1]:
!pip install selenium
!apt-get update

     |████████████████████████████████| 968 kB 4.3 MB/s 
     |████████████████████████████████| 138 kB 64.1 MB/s 
     |████████████████████████████████| 359 kB 52.6 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 4.0 MB 56.2 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-

In [2]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver  /usr/bin
!pip install lxml

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 52 not upgraded.
Need to get 88.4 MB of archives.
After this operation, 297 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 100.0.4896.127-0ubuntu0.18.04.1 [1,142 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser a

In [5]:
# url = "http://www.encar.com/dc/dc_carsearchlist.do?carType=kor&searchType=model&TG.R=A#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.Y.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A20%7D"


# 셀레니움 옵션
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import numpy as np
import time
import datetime

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--start-maximized')

driver = webdriver.Chrome("chromedriver", options=options) # chromedriver가 있는 폴더 경로

# url 파싱
# driver.get(url)
# # 2초 기다려주기
# time.sleep(1)

# 만약에 팝업창 있으면 닫기
# driver.switch_to.window(driver.window_handles[1])
# driver.close()


def car_information():

  col = ['차량이름', '연식', '주행거리', '가격']
  car_information_df = pd.DataFrame(columns=col)

  car_name = []
  car_year = [] 
  distance_driven = []
  price = []

  # 차량이름
  for i in range(1, 22):
    name = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/a')
    car_name.append(name.text)

  # 연식
  for i in range(1, 22):
    year = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/span[1]/span[1]')
    car_year.append(year.text)

  # 주행거리
  for i in range(1, 22):
    driven = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[2]/span[1]/span[2]')
    distance_driven.append(driven.text)

  # 가격
  for i in range(1, 22):
    pri = driver.find_element_by_xpath(f'//*[@id="sr_normal"]/tr[{i}]/td[3]')
    price.append(pri.text)
  
  # 데이터 프레임에 객체 넣기
  car_information_df['차량이름'] = car_name
  car_information_df['연식'] = car_year
  car_information_df['주행거리'] = distance_driven
  car_information_df['가격'] = price

  # 빈 행 제거
  car_information_df['가격'].replace('', np.nan, inplace=True)
  car_information_df.dropna(subset=['가격'], inplace=True)

  return car_information_df

def all_page():

  car_information_df = pd.DataFrame()

  try:
    for i in range(1, 600):
      url = f"http://www.encar.com/fc/fc_carsearchlist.do?carType=for&searchType=model&TG.R=B#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.N.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A{i}%2C%22limit%22%3A20%7D"
      # url 파싱
      driver.get(url)
      # 2초 기다려주기
      time.sleep(1)
      car_information_df = car_information_df.append(car_information(), sort=False, ignore_index=True)
      print(i, 'page crawling...')

  except Exception as e:
    print(e)

  finally:
    return car_information_df


def save_csv(df):
    today = str(datetime.date.today())
    outputFileName = today + 'car_information.csv'
    df.to_csv(outputFileName)
    return

all = all_page()

save_csv(all)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


1 page crawling...
2 page crawling...
3 page crawling...
4 page crawling...
5 page crawling...
6 page crawling...
7 page crawling...
8 page crawling...
9 page crawling...
10 page crawling...
11 page crawling...
12 page crawling...
13 page crawling...
14 page crawling...
15 page crawling...
16 page crawling...
17 page crawling...
18 page crawling...
19 page crawling...
20 page crawling...
21 page crawling...
22 page crawling...
23 page crawling...
24 page crawling...
25 page crawling...
26 page crawling...
27 page crawling...
28 page crawling...
29 page crawling...
30 page crawling...
31 page crawling...
32 page crawling...
33 page crawling...
34 page crawling...
35 page crawling...
36 page crawling...
37 page crawling...
38 page crawling...
39 page crawling...
40 page crawling...
41 page crawling...
42 page crawling...
43 page crawling...
44 page crawling...
45 page crawling...
46 page crawling...
47 page crawling...
48 page crawling...
49 page crawling...
50 page crawling...
51 page c